In [1]:
!pip install roboflow

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 1: pip: command not found


In [3]:
# YOLOv8 Furniture Detection Training
# This notebook walks through the process of training and exporting a YOLOv8 model for furniture detection


# Import necessary libraries
import os
import yaml
from ultralytics import YOLO
from roboflow import Roboflow
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import time

# Check for GPU availability
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of available GPUs: {device_count}")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"GPU {i} memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")

# Load Roboflow dataset (use your API key)
rf = Roboflow(api_key="kSZm10tyRftk8dYF4iTM")  # Replace with your actual API key
project = rf.workspace("object-detection-paxjm").project("furniture-detection-qiufc-czaes")
version = project.version(1)
dataset = version.download("yolov8")
dataset_path = dataset.location

# Print dataset structure
print("\nDataset Information:")
with open(os.path.join(dataset_path, 'data.yaml'), 'r') as f:
    data_config = yaml.safe_load(f)
    print(f"Classes: {data_config['names']}")
    print(f"Number of classes: {data_config['nc']}")

# Verify train/val/test splits
print(f"Train images: {len(list(Path(data_config['train']).glob('*.jpg')))}")
print(f"Validation images: {len(list(Path(data_config['val']).glob('*.jpg')))}")
if 'test' in data_config:
    print(f"Test images: {len(list(Path(data_config['test']).glob('*.jpg')))}")

# Define a function to plot training results
def plot_results(results_file):
    # Load results from the CSV file
    from pandas import read_csv
    results = read_csv(results_file)

    # Create subplots
    fig, ax = plt.subplots(2, 2, figsize=(16, 12))

    # Plot metrics
    ax[0, 0].plot(results['epoch'], results['train/box_loss'], label='train')
    ax[0, 0].plot(results['epoch'], results['val/box_loss'], label='val')
    ax[0, 0].set_title('Box Loss')
    ax[0, 0].legend()

    ax[0, 1].plot(results['epoch'], results['train/cls_loss'], label='train')
    ax[0, 1].plot(results['epoch'], results['val/cls_loss'], label='val')
    ax[0, 1].set_title('Class Loss')
    ax[0, 1].legend()

    ax[1, 0].plot(results['epoch'], results['train/dfl_loss'], label='train')
    ax[1, 0].plot(results['epoch'], results['val/dfl_loss'], label='val')
    ax[1, 0].set_title('DFL Loss')
    ax[1, 0].legend()

    ax[1, 1].plot(results['epoch'], results['metrics/precision(B)'], label='precision')
    ax[1, 1].plot(results['epoch'], results['metrics/recall(B)'], label='recall')
    ax[1, 1].plot(results['epoch'], results['metrics/mAP50(B)'], label='mAP50')
    ax[1, 1].plot(results['epoch'], results['metrics/mAP50-95(B)'], label='mAP50-95')
    ax[1, 1].set_title('Performance Metrics')
    ax[1, 1].legend()

    plt.tight_layout()
    plt.show()

# Model configuration
# Choose model size from: n (nano), s (small), m (medium), l (large), x (xlarge)
MODEL_SIZE = 's'  # adjust based on your GPU VRAM - medium is a good balance
EPOCHS = 50 # 100
IMG_SIZE = 640
BATCH_SIZE = 16  # adjust based on your GPU VRAM
PATIENCE = 15  # for early stopping

# Initialize model
print(f"\nInitializing YOLOv8{MODEL_SIZE} model...")
model = YOLO(f'yolov8{MODEL_SIZE}.pt')  # Load pretrained model

# Define training configuration
training_args = {
    'data': os.path.join(dataset_path, 'data.yaml'),
    'epochs': EPOCHS,
    'imgsz': IMG_SIZE,
    'batch': BATCH_SIZE,
    'patience': PATIENCE,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 4,  # number of worker threads for data loading
    'project': 'furniture_detection',
    'name': f'yolov8{MODEL_SIZE}_furniture',
    'exist_ok': True,  # overwrite existing project/name
    'pretrained': True,  # use pretrained model
    'optimizer': 'auto',  # optimizer (SGD, Adam, AdamW, etc.)
    'lr0': 0.01,  # initial learning rate
    'lrf': 0.01,  # final learning rate fraction
    'momentum': 0.937,  # SGD momentum/Adam beta1
    'weight_decay': 0.0005,  # optimizer weight decay
    'warmup_epochs': 3.0,  # warmup epochs
    'warmup_momentum': 0.8,  # warmup initial momentum
    'warmup_bias_lr': 0.1,  # warmup initial bias lr
    'seed': 0,  # deterministic seed
    'deterministic': True,  # deterministic training
    'save': True,  # save checkpoints
    'save_period': -1,  # save checkpoint every x epochs (-1 to disable)
    'verbose': True,  # verbose output
}

# Start training
print("\nStarting training...")
start_time = time.time()
results = model.train(**training_args)
training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

# Plot training results
results_file = list(Path(f'furniture_detection/yolov8{MODEL_SIZE}_furniture').glob('results*.csv'))[0]
plot_results(results_file)

# Evaluate model on validation data
print("\nEvaluating model on validation data...")
val_results = model.val(data=os.path.join(dataset_path, 'data.yaml'))
print(f"Validation mAP50: {val_results.box.map50:.4f}")
print(f"Validation mAP50-95: {val_results.box.map:.4f}")

# Export model to different formats
print("\nExporting model to different formats...")

# 1. Export to ONNX format
print("Exporting to ONNX...")
onnx_path = model.export(format="onnx", dynamic=True, simplify=True)
print(f"ONNX model saved to: {onnx_path}")

# 2. Export to TorchScript format
print("Exporting to TorchScript...")
torchscript_path = model.export(format="torchscript")
print(f"TorchScript model saved to: {torchscript_path}")

# 3. Export to CoreML format (for macOS/iOS deployment)
try:
    print("Exporting to CoreML...")
    coreml_path = model.export(format="coreml")
    print(f"CoreML model saved to: {coreml_path}")
except Exception as e:
    print(f"CoreML export failed: {e}")

# Optional: Test inference on a sample image
print("\nTesting inference on a sample image...")
sample_images = list(Path(data_config['val']).glob('*.jpg'))
if sample_images:
    test_img = str(sample_images[0])
    print(f"Running inference on {test_img}")

    # Perform detection
    results = model(test_img)

    # Plot results
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(plt.imread(test_img))

    # Get detections
    boxes = results[0].boxes
    for box in boxes:
        # Get coordinates
        x1, y1, x2, y2 = box.xyxy[0].tolist()

        # Get confidence and class
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        label = f"{data_config['names'][cls]} {conf:.2f}"

        # Plot bounding box
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

        # Add label
        ax.text(x1, y1, label, bbox=dict(facecolor='white', alpha=0.7))

    plt.title("Sample Detection")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

print("\nDone! Your model has been trained and exported successfully.")
print(f"Best model path: {model.best}")

# Final reminders and next steps
print("\nReminders for Django integration:")
print("1. Copy the exported ONNX model to your Django app's ml_models directory")
print("2. Update the model_loader.py file to point to the correct model file")
print("3. Test the API with sample images before deploying to production")

GPU available: True
Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3070 Laptop GPU
GPU 0 memory: 8.59 GB
loading Roboflow workspace...
loading Roboflow project...

Dataset Information:
Classes: ['Bed', 'Chair', 'Cupboard', 'Nightstand', 'Sideboard', 'Sofa', 'Table']
Number of classes: 7
Train images: 0
Validation images: 0
Test images: 0

Initializing YOLOv8s model...


100%|██████████| 21.5M/21.5M [00:01<00:00, 11.4MB/s]



Starting training...
Ultralytics 8.3.119 🚀 Python-3.12.9 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/mnt/e/IT/AI/JupyterProject/furniture-detection-1/data.yaml, epochs=50, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=furniture_detection, name=yolov8s_furniture, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

100%|██████████| 755k/755k [00:00<00:00, 4.90MB/s]

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                


...

                   all        652        711      0.911      0.889      0.944      0.783
                   Bed         70         71      0.972      0.887      0.944      0.773
                 Chair         66         76      0.875      0.816      0.908      0.764
              Cupboard        141        154      0.833      0.838      0.903      0.786
            Nightstand         13         13      0.786          1      0.995      0.779
             Sideboard         49         50      0.956        0.9       0.95      0.792
                  Sofa        234        238      0.979      0.969      0.988      0.827
                 Table        106        109      0.974      0.817      0.922      0.761
Speed: 0.5ms preprocess, 4.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to furniture_detection/yolov8s_furniture
Validation mAP50: 0.9443
Validation mAP50-95: 0.7834

Exporting model to different formats...
Exporting to ONNX...
Ultralytics 8.3.119 🚀 Python-3.12.9 

/bin/sh: 1: pip: not found


WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim>=0.1.46" "onnxruntime-gpu" ' returned non-zero exit status 127.
WARNING ⚠️ requirements: ❌ Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim>=0.1.46" "onnxruntime-gpu" ' returned non-zero exit status 127.
ERROR ❌ ONNX: export failure 2.1s: No module named 'onnx'


/bin/sh: 1: pip: not found


ModuleNotFoundError: No module named 'onnx'

In [ ]:


from roboflow import Roboflow
rf = Roboflow(api_key="kSZm10tyRftk8dYF4iTM")
project = rf.workspace("object-detection-paxjm").project("furniture-detection-qiufc-czaes")
version = project.version(1)
dataset = version.download("yolov8")


In [4]:
# 2. Export to TorchScript format
print("Exporting to TorchScript...")
torchscript_path = model.export(format="torchscript")
print(f"TorchScript model saved to: {torchscript_path}")

Exporting to TorchScript...
Ultralytics 8.3.119 🚀 Python-3.12.9 torch-2.7.0+cu126 CPU (AMD Ryzen 7 5800H with Radeon Graphics)

PyTorch: starting from 'furniture_detection/yolov8s_furniture/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (21.5 MB)

TorchScript: starting export with torch 2.7.0+cu126...
TorchScript: export success ✅ 1.8s, saved as 'furniture_detection/yolov8s_furniture/weights/best.torchscript' (42.9 MB)

Export complete (2.2s)
Results saved to /mnt/e/IT/AI/JupyterProject/furniture_detection/yolov8s_furniture/weights
Predict:         yolo predict task=detect model=furniture_detection/yolov8s_furniture/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=furniture_detection/yolov8s_furniture/weights/best.torchscript imgsz=640 data=/mnt/e/IT/AI/JupyterProject/furniture-detection-1/data.yaml  
Visualize:       https://netron.app
TorchScript model saved to: furniture_detection/yolov8s_furniture/weights/be

In [5]:
# 1. Export to ONNX format
print("Exporting to ONNX...")
onnx_path = model.export(format="onnx", dynamic=True, simplify=True)
print(f"ONNX model saved to: {onnx_path}")


Exporting to ONNX...
Ultralytics 8.3.119 🚀 Python-3.12.9 torch-2.7.0+cu126 CPU (AMD Ryzen 7 5800H with Radeon Graphics)

PyTorch: starting from 'furniture_detection/yolov8s_furniture/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.46', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim>=0.1.46" "onnxruntime-gpu" ' returned non-zero exit status 127.


/bin/sh: 1: pip: not found


WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim>=0.1.46" "onnxruntime-gpu" ' returned non-zero exit status 127.
WARNING ⚠️ requirements: ❌ Command 'pip install --no-cache-dir "onnx>=1.12.0" "onnxslim>=0.1.46" "onnxruntime-gpu" ' returned non-zero exit status 127.
ERROR ❌ ONNX: export failure 2.1s: No module named 'onnx'


/bin/sh: 1: pip: not found


ModuleNotFoundError: No module named 'onnx'

In [15]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

# Perform detection
results = model(test_img)

# Your results is a list -> get the first item
result = results[0]

# Read image
img = cv2.imread(test_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Plot image
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(img)

# Now get boxes from result
boxes = result.boxes

for box in boxes:
    # Get coordinates
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

    # Get confidence and class
    conf = box.conf[0].cpu().item()
    cls = int(box.cls[0].cpu().item())
    label = f"{data_config['names'][cls]} {conf:.2f}"

    # Draw bounding box
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                              linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)

    # Draw label
    ax.text(x1, y1, label, fontsize=10, color='black',
            bbox=dict(facecolor='white', alpha=0.7))

plt.title("Sample Detection")
plt.axis('off')
plt.tight_layout()
plt.show()




image 1/1 /mnt/e/IT/AI/JupyterProject/rustic_153.jpg: 640x640 1 Bed, 1 Nightstand, 93.9ms
Speed: 4.2ms preprocess, 93.9ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>